![Title](Images/cisco.png)

# Lab - Internet Meter Data Analysis 

### Objectives
<li>**Part 1: Collect and Store Data**</li>
<li>**Part 2: Manipulate Data**</li>
### Scenario/Background
In this lab, you will acquire Internet speed statistics and store the live data in a comma separated values (csv) file. You will also load the stored data from the csv file to a Python data structure, the Pandas `DataFrame`, and use its functionalities to explore the data and manipulate it so that it is easily readable.
### Required Resources
* 1 PC with Internet access
* Raspberry Pi version 2 or higher
* Python libraries: datetime, csv, subprocess, pandas, numpy
* Datafiles: rpi_data_long.csv

## Part 1: Collect and Store Data
<p>The goal of this first part of the lab is to gather internet speed measurements through the Raspberry Pi. Three kinds of measurements will be collected:
1. Ping speed
2. Download speed
3. Upload speed

#### Step 1: Install Speedtest and Import Python Libraries.
In this step, you will install Speedtest and import Python libraries.
<p>Speedtest-cli is a Python script that measures the upload and download speed of your Internet connection. For more information about speedtest, go to https://github.com/sivel/speedtest-cli.

a) Install `speedtest-cli`.

In [1]:
# Code cell 1
!pip install speedtest-cli


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


This cli allows the Jupyter notebook to connect to the website and store the data.

b) Import the necessary Python libraries.

In [2]:
# Code cell 2
# Python library to manage date and time data
import datetime
# Python library to read and write csv files
import csv
# Python library to execute bash commands from the notebook.
# If you want to know more about this, check this resource: 
# http://www.pythonforbeginners.com/os/subprocess-for-system-administrators
import subprocess

#### Step 2: Generate timestamps using the `datetime` package.
In this lab, measurements of Internet speed statistics will be generated. A crucial step in data acquisition for the majority of data analytics applications, is to associate a timestamp to measurements. 

a) To generate a timestamp, use the `datetime.now` function of the `datetime` package: 

In [3]:
# Code cell 3
date_time = datetime.datetime.now()
print(date_time, type(date_time))

2024-09-11 21:58:42.171455 <class 'datetime.datetime'>


b) An instance of the class `datetime` cannot be directly written to in text form. The function `strftime` parses the date information into a string. The arguments of this function determine the format of the output sting. A description of these parameters can be found in the documentation of the `strftime` function at https://docs.python.org/2/library/time.html. 

In [4]:
# Code cell 4
date_time.strftime('%a, %d %b %Y %H:%M:%S')

'Wed, 11 Sep 2024 21:58:42'

After reading the documentation of the `strftime` function, generate a timestamp and parse it into a string with the following format: YYYY-MM-DD HH:MM:SS.

In [5]:
# Code cell 5
date_time.strftime('%Y-%m-%d %H:%M:%S')


'2024-09-11 21:58:42'

#### Step 3: Run the process and collect the output with Python.

The `speedtest-cli` command, if run from a terminal, returns a string with download and upload speeds. To run the command from this notebook, it is necessary to use the Python module `subprocess`, which allows running a process directly from the notebook code cell. 

a) Run a speed test using the `speedtest-cli` command from Python. The output will be stored in the process_output variable.

In [6]:
# Code cell 6
# This string contains the command line to interface with speedtest.net
speedtest_cmd = "speedtest-cli --simple"
# Execute the process
process = subprocess.Popen(speedtest_cmd.split(), stdout=subprocess.PIPE)
# Collect the command output
process_output = process.communicate()[0]

b) Print the process output. Notice the type for the `process_output` variable.

In [7]:
# Code cell 7
print(process_output, type(process_output))

b'Ping: 6.255 ms\r\nDownload: 0.00 Mbit/s\r\nUpload: 235.81 Mbit/s\r\n' <class 'bytes'>


c) The speed test result is split, and a timestamp is appended to the results.

In [8]:
# Code cell 8
# Store the time at which the speedtest was executed
date_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
process_output = process_output.split()
process_output.append(date_time)
print(process_output, type(process_output))

[b'Ping:', b'6.255', b'ms', b'Download:', b'0.00', b'Mbit/s', b'Upload:', b'235.81', b'Mbit/s', '2024-09-11 21:58:59'] <class 'list'>


d) The speedtest() function is created to return the results from the speedtest-cli command.

In [9]:
# Code cell 9
# function to excute the speed test
def speedtest():
    # We need to store the time at which the speedtest was executed
    date_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # This is a string that contains what we would write on the command line 
    #to interface with speedtest.net
    speedtest_cmd = "speedtest --simple"
    # We now execute the process: 
    process = subprocess.Popen(speedtest_cmd.split(), stdout=subprocess.PIPE)
    process_output = process.communicate()[0]
    process_output = process_output.split()
    # and we add the date and time 
    process_output.append(date_time)
    return process_output

What does the speedtest() function return? What is the code to view results from the speedtest() function?

The speedtest() function executes a speed test using the speedtest-cli tool, captures the output, and appends the current timestamp when the test was run. It uses the subprocess.Popen() method to run the speed test command and stores the output as a list of values, including ping, download speed, and upload speed. The final value in the list is a timestamp in the format YYYY-MM-DD HH:MM:SS, marking when the test occurred. To view the results of the function, you can call result = speedtest() and print the result list, which will display the speed test data (ping, download, upload) and the timestamp in a single list.

In [10]:
# Code cell 10 
# Code to view the results from speedtest() function
process_output

[b'Ping:',
 b'6.255',
 b'ms',
 b'Download:',
 b'0.00',
 b'Mbit/s',
 b'Upload:',
 b'235.81',
 b'Mbit/s',
 '2024-09-11 21:58:59']

#### Step 4: Store the output of the `speedtest()` function.

The comma separated values (csv) is the most common import and export format for spreadsheets and databases. To learn more information about working with csv in Python, navigate to https://docs.python.org/2/library/csv.html.

a) Create a file named test.txt in the /tmp directory and write "test_msg" in the file.

In [11]:
# Code cell 11
with open("./tmp/test.txt",'w') as f:
    f.write('test_msg')

b) Use the Linux command `cat` to verify the creation and content of the file.

In [12]:
# Code cell 12
# Python code to read the content of a file
with open('./tmp/test.txt', 'r') as file:
    content = file.read()
    print(content)


test_msg


c) To check that the file was successfully open:

In [13]:
# Code cell 13
with open("./tmp/test.txt",'r') as f:
   str = f.read()
print(str)

test_msg


d) Understanding the meaning of the `with` statement, especially in combination with `try` and `except` is not required for the rest of this lab, but a useful resource about this is http://effbot.org/zone/python-with-statement.htm.
<p>
To write into a `csv` file, it is necessary to create a `csv.writer` object. Check https://docs.python.org/2/library/csv.html and find out which function of the 'csv.writer' object can be used to add a row to a `csv` file.

In [14]:
# Code cell 14
# function to save data to csv
def save_to_csv(data, filename):
    try:
        # If the file exists, we want to append a new line to it, with the 
        #results of the current experiment
        with open(filename + '.csv', 'a') as f:
            wr = csv.writer(f)
            wr.writerow(data)
    except:
        # If it does not exist, create the file first
        with open(filename + '.csv', 'w') as f:
            # Hint: This is similar to appending new lines to a file.
            # Create a csv writer object
            wr = csv.writer(f)
            # Save (write) to file
            wr.writerow(data)

#### Step 5: Check the collected data.
Write a function to open a csv file and print its content to screen. You can find an example in the 13.1.5 section of https://docs.python.org/2/library/csv.html

In [15]:
# Code cell 15
def print_from_csv(filename): 
    with open(filename + '.csv', 'r') as f:
            re = csv.reader(f)
            # 1. Loop over the rows
            for row in re:
            # 2. print
                print(row)

Now, all the functions needed to collect and store Internet speed data are finished.
#### Step 6: Run the Speedtest multiple times and store the data.
a) Write a `for` loop that calls the speedtest 5 times, prints the output of the tests, and stores the data in a csv file.

In [16]:
# Code cell 16
for i in range(5):
    speedtest_output = speedtest()
    print('Test number {}'.format(i))
    print(speedtest_output)
    save_to_csv(speedtest_output, './tmp/rpi_data_test')
    

Test number 0
[b'Ping:', b'10.793', b'ms', b'Download:', b'295.77', b'Mbit/s', b'Upload:', b'291.18', b'Mbit/s', '2024-09-11 21:59:00']
Test number 1
[b'Ping:', b'7.57', b'ms', b'Download:', b'542.22', b'Mbit/s', b'Upload:', b'314.58', b'Mbit/s', '2024-09-11 21:59:15']
Test number 2
[b'Ping:', b'9.577', b'ms', b'Download:', b'547.92', b'Mbit/s', b'Upload:', b'389.75', b'Mbit/s', '2024-09-11 21:59:32']
Test number 3
[b'Ping:', b'7.549', b'ms', b'Download:', b'304.25', b'Mbit/s', b'Upload:', b'289.81', b'Mbit/s', '2024-09-11 22:00:10']
Test number 4
['2024-09-11 22:00:29']


b) Display the file to verify that the data has been saved correctly.

In [17]:
# Code cell 17
print_from_csv('./tmp/rpi_data_test')


["b'Ping:'", "b'13.045'", "b'ms'", "b'Download:'", "b'5.61'", "b'Mbit/s'", "b'Upload:'", "b'2.30'", "b'Mbit/s'", '2017-07-07 11:37:00']
[]
["b'Ping:'", "b'34.637'", "b'ms'", "b'Download:'", "b'5.45'", "b'Mbit/s'", "b'Upload:'", "b'1.25'", "b'Mbit/s'", '2017-07-07 11:37:26']
[]
["b'Ping:'", "b'13.103'", "b'ms'", "b'Download:'", "b'2.92'", "b'Mbit/s'", "b'Upload:'", "b'0.87'", "b'Mbit/s'", '2017-07-07 11:37:59']
[]
["b'Ping:'", "b'15.088'", "b'ms'", "b'Download:'", "b'4.91'", "b'Mbit/s'", "b'Upload:'", "b'0.87'", "b'Mbit/s'", '2017-07-07 11:38:33']
[]
["b'Ping:'", "b'26.148'", "b'ms'", "b'Download:'", "b'3.81'", "b'Mbit/s'", "b'Upload:'", "b'0.72'", "b'Mbit/s'", '2017-07-07 11:39:15']
[]
["b'Ping:'", "b'11.996'", "b'ms'", "b'Download:'", "b'2.83'", "b'Mbit/s'", "b'Upload:'", "b'0.70'", "b'Mbit/s'", '2017-07-07 17:31:16']
[]
["b'Ping:'", "b'18.365'", "b'ms'", "b'Download:'", "b'7.03'", "b'Mbit/s'", "b'Upload:'", "b'1.37'", "b'Mbit/s'", '2017-07-07 17:32:00']
[]
["b'Ping:'", "b'14.503'", "

If a bigger dataset is needed, the speedtest can be running in the background for more samples.

How would you change the code if you wanted to run the code 100 times?

To modify the code to run the speed test 100 times, you just need to adjust the range(5) to range(100) in the loop. 

In [18]:
# Code cell 18
# Code to run 100 times
# for i in xrange(100):
#     speedtest_output = speedtest()
#     print 'Test number: {}'.format(i)
#     print speedtest_output
#     save_to_csv(speedtest_output, '/tmp/rpi_data')

## Part 2: Manipulate Data

The Python library `pandas` is very useful for working with structured data. The full documentation can be found here: http://pandas.pydata.org/pandas-docs/version/0.14.1/</font>

A larger dataset collected in advance will be used for this part of the lab. The filename is `rpi_data_long.csv`.

#### Step 1: Import the Python libraries.

Import `pandas` and the other libraries used for the next tasks.

In [19]:
# Code cell 19
import datetime
import csv
import pandas as pd
# NumPy is a library that adds support for large, multi-dimensional arrays and matrices
# along with high-level mathematical functions to operate on these arrays
import numpy as np

#### Step 2: Load the `csv` file into a `DataFrame` object using `pandas`.

A `pandas DataFrame` is a 2-dimensional labeled data structure with columns of potentially different types. You can think of it like a spreadsheet or SQL table. The pandas library function `read_csv` automatically converts a `csv` file into a `DataFrame` object.

Read the `read_csv` documentation in http://pandas.pydata.org/pandas-docs/version/0.14.1/generated/pandas.read_csv.html.
This function contains a lot of parameters. The only non-optional one is the `filepath`, i.e. the location of the `csv` file. All the other parameters are optional.

In this step, you will import and view the content of the csv file, `rpi_data_long.csv`. This csv file is located in the same directory as this Jupyter notebook.

a) Assign the file `rpi_data_long.csv` to the variable `data_file`.

In [20]:
# Code cell 20
data_file = './Data/rpi_data_long.csv'

b) Use the Linux command `head` to view the first 10 lines of the csv file.

In [21]:
# Code cell 21
def print_first_n_lines(filename, n=5):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            if i >= n:
                break
            print(row)

# Call the function with your CSV file
print_first_n_lines('./Data/rpi_data_long.csv', 5)

['Ping:', '26.992', 'ms', 'Download:', '91.80', 'Mbit/s', 'Upload:', '14.31', 'Mbit/s', '2016-11-24 13:36:25']
['Ping:', '24.532', 'ms', 'Download:', '88.19', 'Mbit/s', 'Upload:', '14.12', 'Mbit/s', '2016-11-24 13:36:55']
['Ping:', '20.225', 'ms', 'Download:', '59.86', 'Mbit/s', 'Upload:', '14.11', 'Mbit/s', '2016-11-24 13:37:25']
['Ping:', '19.332', 'ms', 'Download:', '91.81', 'Mbit/s', 'Upload:', '14.22', 'Mbit/s', '2016-11-24 13:37:57']
['Ping:', '22.494', 'ms', 'Download:', '92.05', 'Mbit/s', 'Upload:', '14.08', 'Mbit/s', '2016-11-24 13:38:27']


c) Use the `names` parameter of the `read_csv` function to specify the name of the `DataFrame` columns.

In [22]:
# Code cell 22
column_names = [ 'Type A', 'Measure A', 'Units A',
                 'Type B', 'Measure B', 'Units B',
                 'Type C', 'Measure C', 'Units C', 
                 'Datetime']

d) Use the `read_csv` function to read from `data_file` and assign `column_names` as the column names in the dataframe.

In [23]:
# Code cell 23
with open(data_file, 'r') as f:
    df_redundant = pd.read_csv(f, names = column_names)

e) The command `head()` displays the first few rows of the dataframe.

In [24]:
# Code cell 24
# You can specify the number of rows you want to print to screen: 
# you do so passing the number as an argument to the function
# (e.g., head(10))
df_redundant.head()

,Type A,Measure A,Units A,Type B,Measure B,Units B,Type C,Measure C,Units C,Datetime
0,Ping:,26.992,ms,Download:,91.80,Mbit/s,Upload:,14.31,Mbit/s,2016-11-24 13:36:25
1,Ping:,24.532,ms,Download:,88.19,Mbit/s,Upload:,14.12,Mbit/s,2016-11-24 13:36:55
2,Ping:,20.225,ms,Download:,59.86,Mbit/s,Upload:,14.11,Mbit/s,2016-11-24 13:37:25
3,Ping:,19.332,ms,Download:,91.81,Mbit/s,Upload:,14.22,Mbit/s,2016-11-24 13:37:57
4,Ping:,22.494,ms,Download:,92.05,Mbit/s,Upload:,14.08,Mbit/s,2016-11-24 13:38:27


What is the code to read the first 20 lines of the csv file?

df_redundant.head(20)

#### Step 3: Create a concise representation.
In this step, you will create a more compact representation using a copy of the data frame `df_redundant`.

a) Copy `df_redundant` into another dataframe called `df_compact` using `copy()`.

In [25]:
# Code cell 25
df_compact = df_redundant.copy()

b) Rename the columns relative to the measures as shown:

    Measure A -> Ping (ms)
    Measure B -> Download (Mbit/s)
    Measure C -> Upload (Mbit/s)

In [26]:
# Code cell 26
df_compact.rename(columns={'Measure A':'Ping (ms)', 
                           'Measure B': 'Download (Mbit/s)',
                           'Measure C': 'Upload (Mbit/s)'}, inplace=True)
df_compact.head(3)

,Type A,Ping (ms),Units A,Type B,Download (Mbit/s),Units B,Type C,Upload (Mbit/s),Units C,Datetime
0,Ping:,26.992,ms,Download:,91.80,Mbit/s,Upload:,14.31,Mbit/s,2016-11-24 13:36:25
1,Ping:,24.532,ms,Download:,88.19,Mbit/s,Upload:,14.12,Mbit/s,2016-11-24 13:36:55
2,Ping:,20.225,ms,Download:,59.86,Mbit/s,Upload:,14.11,Mbit/s,2016-11-24 13:37:25


c) Because the Types and Units columns are no longer necessary, these columns can be dropped.

In [27]:
# Code cell 27
df_compact.drop(['Type A', 'Type B', 'Type C',
         'Units A', 'Units B', 'Units C'], axis=1, inplace=True)
df_compact.head()

,Ping (ms),Download (Mbit/s),Upload (Mbit/s),Datetime
0,26.992,91.80,14.31,2016-11-24 13:36:25
1,24.532,88.19,14.12,2016-11-24 13:36:55
2,20.225,59.86,14.11,2016-11-24 13:37:25
3,19.332,91.81,14.22,2016-11-24 13:37:57
4,22.494,92.05,14.08,2016-11-24 13:38:27


In the table above, the `Datetime` field is a string. Pandas and Python offer a number of operations to work with date and time that can be very helpful.

In the next step, the string in the `Datetime` column will be separated into two new columns.

#### Step 4: Separate data into two columns.
In this step, you will use Pandas to generate the columns `Date` and `Time` from the column `Datetime` and then drop the `Datetime` column.
<p>The `lambda` function is used create two anonymous functions that extract only the date and the time from a `datetime` object, respectively. Then, use the `pandas` function `apply` to apply this function to an entire column (in practice, `apply` implicitly defines a `for` loop and passes the rows one by one to our `lambda` function). Store the result of the `apply` functions in two new columns of the `DataFrame`.

a) Apply the `lambda` function to iterate through the data frame to split the date from the `Datetime` column.

In [28]:
# Code cell 28
df_compact['Date'] = df_compact['Datetime'].apply(lambda dt_str: pd.to_datetime(dt_str).date())

b) Repeat step a to split time from `Datetime` column. 

In [29]:
# Code cell 29
# Please note, this requires an intermediate step, because of how NaT are treated by the time() function.
# Reference: https://github.com/pandas-dev/pandas/issues/11453
temp = df_compact['Datetime'].apply(lambda dt_str: pd.to_datetime(dt_str))
df_compact['Time'] = temp.dt.time

c) All the information for the `Datetime` column has now been copied to the `Date` and `Time` columnns. The `Datetime` column serves no purpose. The `Datetime` column can be dropped from the data frame.
<p>Enter the code to drop the `Datetime` column in the cell below.

In [30]:
# Code cell 30
df_compact = df_compact.drop(columns=['Datetime'])

Enter the code to print out the first 3 rows of the data frame to verify the changes.

In [31]:
# Code cell 31
df_compact.head(3)

,Ping (ms),Download (Mbit/s),Upload (Mbit/s),Date,Time
0,26.992,91.80,14.31,2016-11-24,13:36:25
1,24.532,88.19,14.12,2016-11-24,13:36:55
2,20.225,59.86,14.11,2016-11-24,13:37:25


d) Use the `type` function to print out the variable type of the values in the `Date` and `Time` columns.

In [32]:
# Code cell 32
print(df_compact['Date'][0], type(df_compact['Date'][0]) )
print(df_compact['Time'][0], type(df_compact['Time'][0]) )

2016-11-24 <class 'datetime.date'>
13:36:25 <class 'datetime.time'>


#### Step 5: Save the new data frame.
Save the pandas dataframe `df_compact` as a csv file called `rpi_data_compact`:

In [33]:
# Code cell 33
df_compact.to_csv('./Data/rpi_data_compact.csv')

<font size='0.5'>&copy; 2017 Cisco and/or its affiliates. All rights reserved. This document is Cisco Public.<font>